In [184]:
from brain_lib import *
from helpful_functions import *

In [185]:
import requests
from urllib.parse import urljoin
import time
import json
import os
import getpass
from pathlib import Path

import pandas as pd

from multiprocessing.pool import ThreadPool
from functools import partial

import tqdm
import random

In [186]:
get_credentials()

('ssnishasis22@iitk.ac.in', 'Hello@2023')

In [188]:
s = start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_6HLZnbUfuGSaFshx6iYdssD2



 f


In [194]:
check_session_timeout(s)

14396.373146

In [195]:
datasets_df = get_datasets(s)
datasets_df.head()

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount
0,analyst11,ESG scores,Environmental Social Governance scores that ex...,analyst,analyst-esg,USA,1,TOP3000,0.6818,None,2.0,182,12190,197
1,analyst14,Estimations of Key Fundamentals,This dataset reports many items from financial...,analyst,analyst-analyst-estimates,USA,1,TOP3000,0.6133,None,1.0,447,103064,870
2,analyst15,Earnings forecasts,This dataset provides bottom-up forecast data ...,analyst,analyst-analyst-estimates,USA,1,TOP3000,0.9892,None,1.0,453,674482,2538
3,analyst16,Real Time Estimates,This dataset provides real-time access to the ...,analyst,analyst-crowdsourced-estimates,USA,1,TOP3000,0.8252,None,2.0,86,2892,107
4,analyst2,Analysts Estimates of Key Fundamentals,This data contains forecasts of key fundamentals.,analyst,analyst-crowdsourced-estimates,USA,1,TOP3000,0.4702,None,2.0,399,14626,27


In [196]:
selected_datasets_df = datasets_df.query("""
        delay == 1 &\
        0. < coverage <= 1 &\
        0 < fieldCount < 1000000000000000 &\
        region == 'USA' &\
        universe == 'TOP3000' &\
        0 <= userCount < 1000000000000000 &\
        0 < valueScore < 10 &\
        name.str.contains('', case=False) == 1 &\
        (category == 'model')
    """, engine='python')
selected_datasets_df

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount
66,model10,Research Indicators,This data includes technical indicators and ra...,model,model-technical-models,USA,1,TOP3000,0.9439,None,2.0,385,79038,92
67,model106,Analysts rating model,The dataset provides ratings and evaluations r...,model,model-estimates-models,USA,1,TOP3000,0.4865,None,2.0,71,700,14
68,model109,Fundamentals and Technical Indicators Model,Alpha model. Normal distributed cross-sectiona...,model,model-valuation-models,USA,1,TOP3000,0.7917,None,2.0,192,11758,298
69,model11,Revenue Forecast Data,This dataset holds a score designed to forecas...,model,model-consumer-models,USA,1,TOP3000,0.5776,None,2.0,94,1666,1
70,model110,Big data and machine learning based model,Machine learning based scores which are catchi...,model,model-mlai-models,USA,1,TOP3000,0.8518,None,2.0,188,4860,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,model57,Quantitative Stock Valuation Model,This dataset is a model dataset created mainly...,model,model-valuation-models,USA,1,TOP3000,0.5388,None,1.0,111,4576,22
128,model7,Value & Growth Alpha Model,This is a model dataset. It includes ranks and...,model,model-valuation-models,USA,1,TOP3000,0.9241,None,1.0,126,4458,24
129,model77,Analysts' Factor Model,Factor model dataset based on anaysts' evaluation,model,model-technical-models,USA,1,TOP3000,0.8073,None,2.0,244,33416,1546
130,model8,Fundamental Analysis Model,This is a model dataset for USA listed compani...,model,model-technical-models,USA,1,TOP3000,0.9679,None,1.0,3306,30948,54


In [197]:
dataset_id = ['other452', 'model130']

In [198]:
datafields_df = [get_datafields(s, dataset_id=x) for x in dataset_id] 

In [199]:
datafields_df[1]

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount
0,mdl130_qes_10day_return,10 Day Return,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.7784,None,4,16
1,mdl130_qes_1day_return,1 Day Return,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.7783,None,4,10
2,mdl130_qes_1month_return,1 Month Return,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.7774,None,3,10
3,mdl130_qes_3month_idio_vol,3 Month Idiosyncratic Volatility (CAPM-Annuali...,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.7640,None,6,24
4,mdl130_qes_5day_return,5 Day Return,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.7784,None,2,6
5,mdl130_qes_5year_return,5 year Return,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.6209,None,3,8
6,mdl130_qes_current_price,Current Price,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.7785,None,2,4
7,mdl130_qes_google_search_interest_within_the_us,Google Search Interest within the US.Higher = ...,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.3363,None,0,0
8,mdl130_qes_google_search_interest_worldwide,Google Search Interest Worldwide. Higher = mor...,"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.5151,None,2,4
9,mdl130_qes_hfc,HF Concentration (%),"{'id': 'model130', 'name': 'Retail Investor Da...",model,model-sentiment,USA,1,TOP3000,MATRIX,0.7339,None,2,6


In [200]:
expression_list = []

In [201]:
days_list = [60,80,100,120]
for a in datafields_df[0]['id'][:10]:
    for b in datafields_df[1]['id'][:10]:
        for c in days_list:
            expression_list.append(f'group_neutralize(if_else(rank(ts_backfill({a},{c}))>0.8 && rank({b})<0.2,-1,0),sector)*rank(anl46_indicator)')

In [202]:
alpha_list = [generate_alpha(x, neutralization = "SLOW_AND_FAST", decay = 5) for x in expression_list]

In [203]:
result = simulate_alpha_list_multi(s, alpha_list)

 43%|████▎     | 57/134 [1:39:39<2:14:37, 104.91s/it]


KeyboardInterrupt: 

In [124]:
result[0]

{'alpha_id': 'jnAzRmZ',
 'simulate_data': {'type': 'REGULAR',
  'settings': {'nanHandling': 'OFF',
   'instrumentType': 'EQUITY',
   'delay': 1,
   'universe': 'TOP3000',
   'truncation': 0.08,
   'unitHandling': 'VERIFY',
   'pasteurization': 'ON',
   'region': 'USA',
   'language': 'FASTEXPR',
   'decay': 0,
   'neutralization': 'INDUSTRY',
   'visualization': False},
  'regular': 'open'},
 'is_stats':        pnl  bookSize  longCount  shortCount  turnover  returns  drawdown  \
 0 -2270487  20000000       1007        2096    0.0162  -0.0231    0.4184   
 
      margin  fitness  sharpe alpha_id  
 0 -0.002854    -0.12   -0.27  jnAzRmZ  ,
 'pnl': None,
 'stats': None,
 'is_tests':                        name   result  limit     value        date  \
 0                LOW_SHARPE     FAIL   1.58 -0.270000         NaN   
 1               LOW_FITNESS     FAIL   1.00 -0.120000         NaN   
 2              LOW_TURNOVER     PASS   0.01  0.016200         NaN   
 3             HIGH_TURNOVER    

In [128]:
result_st1 = prettify_result(result, detailed_tests_view=True)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,...,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_competition,matches_themes,prod_correlation,regular_submission,self_correlation
0,-2270487,20000000,1007,2096,0.0162,-0.0231,0.4184,-0.002854,-0.12,-0.27,...,"{'limit': 1.58, 'result': 'FAIL', 'value': -1.16}","{'limit': 1.0, 'result': 'FAIL', 'value': -0.12}","{'limit': 1.58, 'result': 'FAIL', 'value': -0.27}","{'limit': -0.12, 'result': 'PASS', 'value': -0...","{'limit': 0.01, 'result': 'PASS', 'value': 0.0...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}"
1,-2345654,20000000,1008,2095,0.0149,-0.0238,0.4184,-0.003196,-0.12,-0.28,...,"{'limit': 1.58, 'result': 'FAIL', 'value': -1.17}","{'limit': 1.0, 'result': 'FAIL', 'value': -0.12}","{'limit': 1.58, 'result': 'FAIL', 'value': -0.28}","{'limit': -0.12, 'result': 'PASS', 'value': -0...","{'limit': 0.01, 'result': 'PASS', 'value': 0.0...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}"
2,-2441452,20000000,1007,2095,0.0157,-0.0248,0.4215,-0.003161,-0.13,-0.29,...,"{'limit': 1.58, 'result': 'FAIL', 'value': -1.18}","{'limit': 1.0, 'result': 'FAIL', 'value': -0.13}","{'limit': 1.58, 'result': 'FAIL', 'value': -0.29}","{'limit': -0.13, 'result': 'PASS', 'value': -0...","{'limit': 0.01, 'result': 'PASS', 'value': 0.0...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}"
